# Stage 2: Display Metadata

Given the vast differences in the geometries between different polygons (and the fact that each one is a boring, boring square, this notebook will enrich it with metadata based on the size of the box?

In [69]:
var turf = require('turf')
var fs = require('fs')
var _ = require('lodash')

undefined

In [2]:
fc = JSON.parse(fs.readFileSync('/data/www/jennings/infovis/image-tweets-by-polygon.geojson'))
fc.features.length;

4525

In [6]:
fc2 = JSON.parse(JSON.stringify(fc))
fc2.features.length

4525

In [64]:
var maxZooms = {
    40000 : 4.5,
    20000 : 8.5,
    10000 : 10
}

undefined

In [74]:
var newFeatures = []

var count=0
var zeroes=0
var zero_feats = 0;
var tweets = 0;
fc2.features.slice(0,1000).forEach(function(feat){
    var area = Math.floor(turf.area(feat.geometry)/1000000)
    //That's square meters
    if (area < 9999999){
//         if (area < 500){
            count++;
            feat.properties.area = Math.floor(area)
            newFeatures.push(feat)
            tweets+= feat.properties.tweets.length
//         }
    }else{
        zeroes ++;
        zero_feats += feat.properties.tweets.length
    }
})
console.log(count, tweets, zeroes, zero_feats)

997 3455 3 69


undefined

In [78]:
newFC = {type: "FeatureCollection",features: _.sortBy(newFeatures,function(f){return -f.properties.area})}
null

null

In [79]:
fs.writeFileSync('/data/www/jennings/infovis/poly-full.geojson', JSON.stringify(newFC))

undefined